In [ ]:
import tensorflow as tf
from matplotlib import pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, Dropout
from tensorflow.keras.models import Sequential
from sklearn.metrics import plot_confusion_matrix

### Run on GPU

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
#@Load CIFAR-10 Dataset
cifar10 = tf.keras.datasets.cifar10

# Distribute it to train and test set
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)


In [ ]:
# Reduce pixel values
x_train, x_test = x_train / 255.0, x_test / 255.0

# flatten the label values
y_train, y_test = y_train.flatten(), y_test.flatten()


In [ ]:
x_valid, x_test, y_valid, y_test = train_test_split(x_test, y_test, train_size=0.5, shuffle=False)

In [ ]:
#@One hot encoding
labels = np.array([0, 1 ,2, 3, 4, 5, 6, 7, 8, 9]).reshape(-1, 1)

# Design one hot encoder
encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder on the labels
encoder.fit(labels)

# Function to one-hot encode data
def one_hot_encode(data):
    return encoder.transform(np.array(data).reshape(-1, 1))


# Function to decode one-hot encoded data
def decode(encoded_data):
    return encoder.inverse_transform(encoded_data).flatten().tolist()

In [ ]:
y_train = one_hot_encode(y_train)
y_valid = one_hot_encode(y_valid)
y_test = one_hot_encode(y_test)

## Phase 1:

VGG Style Conv layers followed by FCNN Layers and classification. 

In [ ]:
#@ Define model
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)))
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(10, activation='softmax'))


model.summary()

### Training

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_history = model.fit(x_train, y_train, validation_data=(x_valid, y_valid), epochs=15)

In [ ]:
# plotting loss vs epoch curve of model
loss = model_history.history['loss']
val_loss = model_history.history['val_loss']
epochs = range(1, len(loss)+1)
plt.figure()
plt.plot(epochs, loss, label='train loss')
plt.plot(epochs, val_loss, label='val loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Find and print model accuracy against test set
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=2)
print('\nTest accuracy:', test_acc)

## Phase 2:

VGG Style Conv layers and Max Pooling layers followed by FCNN classifier.

In [ ]:
#@ Define model
model_maxpool = Sequential()
model_maxpool.add(Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)))
model_maxpool.add(Conv2D(32, (3,3), activation='relu'))
model_maxpool.add(MaxPooling2D(2,2))
model_maxpool.add(Conv2D(64, (3,3), activation='relu'))
model_maxpool.add(Conv2D(64, (3,3), activation='relu'))
model_maxpool.add(MaxPooling2D(2,2))
model_maxpool.add(Conv2D(128, (3,3), activation='relu'))
model_maxpool.add(Conv2D(128, (3,3), activation='relu'))
model_maxpool.add(Flatten())
model_maxpool.add(Dense(100, activation='relu'))
model_maxpool.add(Dense(10, activation='softmax'))

model_maxpool.summary()

### Training

In [ ]:
model_maxpool.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_history = model_maxpool.fit(x_train, y_train, validation_data=(x_valid, y_valid), epochs=15)

In [ ]:
# plotting loss vs epoch curve of model
loss = model_history.history['loss']
val_loss = model_history.history['val_loss']
epochs = range(1, len(loss)+1)
plt.figure()
plt.plot(epochs, loss, label='train loss')
plt.plot(epochs, val_loss, label='val loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Find and print model accuracy against test set
test_loss, test_acc = model_maxpool.evaluate(x_test,  y_test, verbose=2)
print('\nTest accuracy:', test_acc)

## Phase 3:

VGG Style Conv, MaxPool and Dropout layers followed by FCNN classifier.

In [ ]:
#@ Define model
model_f = Sequential()
model_f.add(Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)))
model_f.add(Conv2D(32, (3,3), activation='relu'))
model_f.add(MaxPooling2D(2,2))
model_f.add(Dropout(0.1))
model_f.add(Conv2D(64, (3,3), activation='relu'))
model_f.add(Conv2D(64, (3,3), activation='relu'))
model_f.add(MaxPooling2D(2,2))
model_f.add(Dropout(0.1))
model_f.add(Conv2D(128, (3,3), activation='relu'))
model_f.add(Conv2D(128, (3,3), activation='relu'))
model_f.add(Dropout(0.1))
model_f.add(Flatten())
model_f.add(Dense(100, activation='relu'))
model_f.add(Dropout(0.1))
model_f.add(Dense(100, activation='relu'))
model_f.add(Dropout(0.2))
model_f.add(Dense(10, activation='softmax'))

model_f.summary()

### Training

In [ ]:
model_f.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_history = model_f.fit(x_train, y_train, validation_data=(x_valid, y_valid), epochs=15)

In [ ]:
# plotting loss vs epoch curve of model
loss = model_history.history['loss']
val_loss = model_history.history['val_loss']
epochs = range(1, len(loss)+1)
plt.figure()
plt.plot(epochs, loss, label='train loss')
plt.plot(epochs, val_loss, label='val loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Find and print model accuracy against test set
test_loss, test_acc = model_f.evaluate(x_test,  y_test, verbose=2)
print('\nTest accuracy:', test_acc)